# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Programa

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

In [2]:
#include <stdio.h>

int main() {
    float parcela = 200;
    int num_parcelas = 5;
    float juros = 1;

    for(int i = 0; i < num_parcelas; i++){
        printf("Parcela %d: %f\n", (i+1), parcela);
        parcela = parcela * (1 + juros/100);
    }
}

Parcela 1: 200.000000
Parcela 2: 202.000000
Parcela 3: 204.020004
Parcela 4: 206.060196
Parcela 5: 208.120789


## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de uma única parcela X do empréstimo. Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa principal. A função deve apenas calcular uma única parcela, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [3]:
#include <stdio.h>

float calculaParcela(float val_inicial, int num_parcela, float juros){
    
    float parcela;
    parcela = val_inicial;
    
    for (int i = 1; i < num_parcela; i++){
        parcela = parcela * (1 + juros/100);
    }
    
    return parcela;
}

int main(){
    float valor_emprestimo = 200;
    int num_parcela_desejada = 5;
    float juros = 1;
    float parcela = 0;
    
    parcela = calculaParcela(valor_emprestimo, num_parcela_desejada, juros);
    
    printf("Valor da %d parcela: %f", num_parcela_desejada, parcela);
    
}

Valor da 5 parcela: 208.120789

## Exercício Parte 3 - Mantendo o Estado

Modifique a função (módulo) `proximaParcela` acima de modo que ela tenha uma memória da situação em que estão as coisas, sem que o `main` precise ficar informando novamente coisas que já informou no início. Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência dos módulos com o programa principal.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente.

In [5]:
#include <stdio.h>

// Variáveis globais para controle do empréstimo e suas parcelas
float val_inicial_global;
int   num_parcelas_global;
float juros_global;
int   atual;
float parcela;

void criaEmprestimo(float val_inicial, int num_parcelas, float juros) {
    val_inicial_global = val_inicial;
    num_parcelas_global = num_parcelas;
    juros_global = juros;
    atual = 1;
    parcela = val_inicial;
}

float proximaParcela() {
    float retorno = parcela;
    atual++;
    if (atual <= num_parcelas_global)
        parcela = parcela * (1 + juros_global/100);
    else
        parcela = 0;
    return retorno;
}

int main() {
    criaEmprestimo(200, 5, 1);
    
    int i = 1;
    float parcela_atual = proximaParcela();
    while (parcela_atual > 0) {
        printf("O valor da parcela %d: %f\n", i, parcela_atual);
        parcela_atual = proximaParcela();
        i++;
    }   
}

O valor da parcela 1: 200.000000
O valor da parcela 2: 202.000000
O valor da parcela 3: 204.020004
O valor da parcela 4: 206.060196
O valor da parcela 5: 208.120789


## Exercício Parte 4 - Minimizando os Parâmetros

Retomando a solução da `Parte 2`, em que a função `main` deve ter usado vários parâmetros para a comunicação com a função `proximaParcela`,  modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar o estado completo do empréstimo.

In [2]:
#include <stdio.h>

typedef struct {
    float valor;
    int   num_parcelas;
    float juros;
    int atual;
    float val_parcela;
} Emprestimo;

Emprestimo criaEmprestimo(float valor, int num_parcelas, float juros) {
    Emprestimo umEmprestimo;

    umEmprestimo.valor = valor;
    umEmprestimo.num_parcelas = num_parcelas;
    umEmprestimo.juros = juros;
    umEmprestimo.atual = 1;
    umEmprestimo.val_parcela = valor;
    
    return umEmprestimo;
}

float proximaParcela(Emprestimo *umEmprestimo) {
    float retorno = umEmprestimo->val_parcela;
    if (umEmprestimo->atual < umEmprestimo->num_parcelas)
        umEmprestimo->val_parcela = umEmprestimo->val_parcela * (1 + umEmprestimo->juros/100);
    else
        umEmprestimo->val_parcela = 0;
    (umEmprestimo->atual)++;
    return retorno;
}

int main() {
    Emprestimo umEmprestimo;
    
    umEmprestimo = criaEmprestimo(200, 5, 1);
    
    int i = 1;
    float parcela = proximaParcela(&umEmprestimo);
    while (parcela > 0) {
        printf("O valor da parcela %d: %f\n", i, parcela);
        i++;
        parcela = proximaParcela(&umEmprestimo);
    }   
}

O valor da parcela 1: 200.000000
O valor da parcela 2: 202.000000
O valor da parcela 3: 204.020004
O valor da parcela 4: 206.060196
O valor da parcela 5: 208.120789


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada empréstimo separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, a função `proximaParcela` ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que, cada novo empréstimo só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo.

In [1]:
#include <stdio.h>

typedef struct {
    float valor;
    int   num_parcelas;
    float juros;
    int atual;
    float val_parcela;
} Emprestimo;

Emprestimo criaEmprestimo(float valor, int num_parcelas, float juros) {
    Emprestimo umEmprestimo;

    umEmprestimo.valor = valor;
    umEmprestimo.num_parcelas = num_parcelas;
    umEmprestimo.juros = juros;
    umEmprestimo.atual = 1;
    umEmprestimo.val_parcela = valor;
    
    return umEmprestimo;
}

float proximaParcela(Emprestimo *umEmprestimo) {
    float retorno = umEmprestimo->val_parcela;
    if (umEmprestimo->atual < umEmprestimo->num_parcelas)
        umEmprestimo->val_parcela = umEmprestimo->val_parcela * (1 + umEmprestimo->juros/100);
    else
        umEmprestimo->val_parcela = 0;
    (umEmprestimo->atual)++;
    return retorno;
}

int main() {
    
    Emprestimo emprestimo1 = criaEmprestimo(200, 5, 1);
    Emprestimo emprestimo2 = criaEmprestimo(500, 7, 2);
    
    int i = 1;
    float parcela1 = proximaParcela(&emprestimo1);
    float parcela2 = proximaParcela(&emprestimo2);
    
    while (parcela1 > 0 || parcela2 > 0) {
        if (parcela1 > 0)
            printf("Emprestimo 1: parcela %d: %f\n", i, parcela1);
        
        if (parcela2 > 0)
            printf("Emprestimo 2: parcela %d: %f\n", i, parcela2);
        
        parcela1 = proximaParcela(&emprestimo1);
        parcela2 = proximaParcela(&emprestimo2);
        i++;
    }   
}

Emprestimo 1: parcela 1: 200.000000
Emprestimo 2: parcela 1: 500.000000
Emprestimo 1: parcela 2: 202.000000
Emprestimo 2: parcela 2: 510.000000
Emprestimo 1: parcela 3: 204.020004
Emprestimo 2: parcela 3: 520.200012
Emprestimo 1: parcela 4: 206.060196
Emprestimo 2: parcela 4: 530.604004
Emprestimo 1: parcela 5: 208.120789
Emprestimo 2: parcela 5: 541.216064
Emprestimo 2: parcela 6: 552.040405
Emprestimo 2: parcela 7: 563.081177
